In [ ]:
import sys
sys.path.append("../GarNet")

import pandas as pd
import numpy as np

import garnet

# Generate GarNet file

Reference file can be generated as seen in Part I of `Construct_hg19_GarNet_File.ipynb`. 

In [ ]:
reference_file = "test_data/reference.normalized.bed"
motif_file = "test_data/motifmap.normalized.cleaned.short.bed"
output_file = "test_data/garnetDB.tsv"

garnet.construct_garnet_file(reference_file, motif_file, output_file, 10000)

# Map peaks to genes

This is accomplished by intersecting motifs with genes specified in the open chromatin bed file. 

In [ ]:
peaks_file = "test_data/diffSites.bed"

df = garnet.map_peaks(peaks_file, output_file)